In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# The below cells are related to arrangement of data so that we can readly consume it for model training

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip', 'r')
zip_ref.extractall('/kaggle/temp/dogs-vs-cats/')
zip_ref = zipfile.ZipFile('/kaggle/input/dogs-vs-cats/test1.zip', 'r')
zip_ref.extractall('/kaggle/temp/dogs-vs-cats/')
zip_ref.close()

In [ ]:
os.listdir('/kaggle/temp/dogs-vs-cats/')

In [ ]:
os.mkdir('/kaggle/temp/dogs-vs-cats/train/dog/')
os.mkdir('/kaggle/temp/dogs-vs-cats/train/cat/')

In [ ]:
import shutil
for file in os.listdir('/kaggle/temp/dogs-vs-cats/train/'):
    if 'dog' in str(file) and 'jpg' in str(file):
        shutil.move('/kaggle/temp/dogs-vs-cats/train/'+str(file),'/kaggle/temp/dogs-vs-cats/train/dog/'+str(file))
    if 'cat' in str(file) and 'jpg' in str(file):
        shutil.move('/kaggle/temp/dogs-vs-cats/train/'+str(file),'/kaggle/temp/dogs-vs-cats/train/cat/'+str(file))

In [ ]:
print(len(os.listdir('/kaggle/temp/dogs-vs-cats/train/dog/')))
print(len(os.listdir('/kaggle/temp/dogs-vs-cats/train/cat/')))

In [ ]:
os.mkdir('/kaggle/temp/dogs-vs-cats/validate/')
os.mkdir('/kaggle/temp/dogs-vs-cats/validate/dog/')
os.mkdir('/kaggle/temp/dogs-vs-cats/validate/cat/')

In [ ]:
os.listdir('/kaggle/temp/dogs-vs-cats/')

In [ ]:
for file in os.listdir('/kaggle/temp/dogs-vs-cats/train/dog/')[:1250]:
    shutil.move('/kaggle/temp/dogs-vs-cats/train/dog/'+str(file),'/kaggle/temp/dogs-vs-cats/validate/dog/'+str(file))
for file in os.listdir('/kaggle/temp/dogs-vs-cats/train/cat/')[:1250]:
    shutil.move('/kaggle/temp/dogs-vs-cats/train/cat/'+str(file),'/kaggle/temp/dogs-vs-cats/validate/cat/'+str(file))

In [ ]:
print(len(os.listdir('/kaggle/temp/dogs-vs-cats/train/dog/')))
print(len(os.listdir('/kaggle/temp/dogs-vs-cats/train/cat/')))
print(len(os.listdir('/kaggle/temp/dogs-vs-cats/validate/dog/')))
print(len(os.listdir('/kaggle/temp/dogs-vs-cats/validate/cat/')))

# The below cells are for defining a TensorFlow model and preparing the Image Generator for feeding it to the model.fit

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAIN_DIR = '/kaggle/temp/dogs-vs-cats/train/'
VALIDATION_DIR ='/kaggle/temp/dogs-vs-cats/validate/'

train_datagen = ImageDataGenerator(rescale=1/255,rotation_range=40, width_shift_range=0.15,
    height_shift_range=0.15,shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, vertical_flip=True)
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    batch_size = 32,
    shuffle=True,
    target_size=(300,300),
    class_mode='binary'
)


validation_datagen = ImageDataGenerator(rescale=1/255)
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    batch_size = 32,
    target_size=(300,300),
    class_mode='binary'
)

In [ ]:
model = tf.keras.models.Sequential(
[
    tf.keras.layers.Conv2D(16,(3,3),input_shape=(300,300,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(train_generator,
                              epochs=500,
                              validation_data=validation_generator,
                              callbacks=[
                                  tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3),
                                  tf.keras.callbacks.ModelCheckpoint(
                                        filepath='/kaggle/working/',
                                        save_weights_only=True,
                                        monitor='val_accuracy',
                                        mode='max',
                                        save_best_only=True)
                              ])

# The below cells visualize the model accuracy and loss to check the performance

In [ ]:
try:
    from matplotlib import pyplot as plt
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
except:
    pass

try:
    from matplotlib import pyplot as plt
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
except:
    pass